In [1]:
# Installment
from urllib.request import urlopen
from bs4 import BeautifulSoup


In [2]:
def ScrapDataItem(url):
    html = urlopen(url).read()
    s = BeautifulSoup(html, "html.parser")
    # print(s.prettify())
    return s

def PrintTupleData(tuple):
    for data in tuple:
        print(data, end=" || ")
    print()

In [4]:
def ScrapProductHrefAndID(url):
    tempSoup = ScrapDataItem(url)
    allProducts = tempSoup.findAll("div", "dne-itemtile dne-itemtile-large")

    hrefArr = []
    for i in allProducts:
        listingID = i['data-listing-id']
        href = i.find("a")['href']
        hrefArr.append((href,listingID))
    return hrefArr

# Usage example
# test = ScrapProductHref("https://www.ebay.com/globaldeals/tech/laptops-netbooks")

In [5]:
def ScrapProductPage(url, type, listingID):    
    tempSoup = ScrapDataItem(url)

    name = tempSoup.find("h1", "x-item-title__mainTitle").find("span", class_="ux-textspans ux-textspans--BOLD").get_text()
    condition = tempSoup.find("div", class_="x-item-condition-value").find("span", class_="ux-textspans").get_text()

    availability = tempSoup.find("div", "d-quantity__availability")
    if (availability is not None):
        availability = availability.findAll("span", "ux-textspans")
        available = availability[0].get_text() # [:-9] #removing "available"
        if (len(availability) > 1): # if there is sold data
            sold = availability[2].get_text()[:-4] # removing "sold"
        else :
            sold = None
    else :
        available = None; sold = None

    product_price_original = tempSoup.find("div", "x-price-primary").find("span", "ux-textspans").get_text() # [4:] #removing "US $"
    product_price_idr = tempSoup.find("div", "x-price-approx").find("span", "ux-textspans ux-textspans--SECONDARY ux-textspans--BOLD").get_text() # [3:] #removing "IDR"
    shipping_price_original = tempSoup.find("div", "ux-labels-values__values-content").find("span", "ux-textspans ux-textspans--BOLD")
    if (shipping_price_original is not None):
        shipping_price_original = shipping_price_original.get_text() # [4:]  # remocing "US $"
    shipping_location = tempSoup.find("div", "ux-labels-values__values-content").find("span", "ux-textspans ux-textspans--SECONDARY")
    if (shipping_location is not None):
        shipping_location = shipping_location.get_text()[12:] # removing "located in: "
    return_info = tempSoup.find("div", "ux-layout-section ux-layout-section--returns").findAll("span", "ux-textspans")[1].get_text()
    seller = tempSoup.find("div", "d-stores-info-categories__container__info__section__title").find("span", "ux-textspans ux-textspans--BOLD").get_text()

    tuple = (listingID, name, type, available, sold, product_price_original, product_price_idr, shipping_price_original, shipping_location, return_info, seller, condition)
    return tuple


In [6]:
prodsArray = [
    # ( categories, url )
    ( "Laptops & Netbooks" , "https://www.ebay.com/globaldeals/tech/laptops-netbooks"),
    ( "Cameras & Photo" , "https://www.ebay.com/globaldeals/tech/cameras-photo"),
    ( "TV, Video & Home Audio" , "https://www.ebay.com/globaldeals/tech/tv-video-home-audio" ),
    ( "iPads, Tablets & eReaders", "https://www.ebay.com/globaldeals/tech/ipads-tablets-ereaders"),
    ( "Video Games & Consoles", "https://www.ebay.com/globaldeals/tech/video-games-consoles"),
    ( "Phone Cases & Accessories", "https://www.ebay.com/globaldeals/tech/phone-cases-accessories"),
    ( "Vehicle Electronics & GPS", "https://www.ebay.com/globaldeals/tech/vehicle-electronics-gps"),
    ( "Memory Drives & Storage", "https://www.ebay.com/globaldeals/tech/memory-drives-storage"),
    ( "Printers & Printer Supplies", "https://www.ebay.com/globaldeals/tech/printers-printer-supplies"),
    ( "Headphones & Portable Audio", "https://www.ebay.com/globaldeals/tech/headphones-portable-audio"),
    ( "Computer Accessories", "https://www.ebay.com/globaldeals/tech/computer-accessories"),
    ( "Other Consumer Electronics", "https://www.ebay.com/globaldeals/tech/more-consumer-electronics")
]

In [7]:
# Scrap Every Data, make it insert it into a big array
dataArr = []
count = 0

for i in prodsArray:
    hrefArr = ScrapProductHrefAndID(i[1])
    for j in hrefArr:
        # print(count)
        tempTuple = ScrapProductPage(j[0], i[0], j[1])
        if (count % 50 == 0):
            print(tempTuple)
        dataArr.append(tempTuple)
        count += 1

('354907151520', 'Microsoft 13" Surface Pro 8 (i5, 8GB RAM, 256GB SSD, Graphite) with Surface Pen', 'Laptops & Netbooks', 'Limited quantity available', '1 ', 'US $669.99', 'IDR9,999,850.75', 'US $54.46 ', 'Avenel, New Jersey, United States', '30 days returns', 'Always Deals Store', 'New')
('204160038683', 'TV Wall Mount Wall Holder LCD LED TV 32-65/70" Tilt Black DE', 'TV, Video & Home Audio', 'More than 10 available', '74 ', 'EUR 14.50', 'IDR243,307.84', None, 'Euskirchen, Germany', '30 days returns', 'delightmark', 'New')
('334818857843', 'Xbox Series S 512GB Digital Console, White - New', 'Video Games & Consoles', 'Limited quantity available', '3 ', 'GBP 239.99', 'IDR4,698,968.38', 'GBP 86.14 ', 'Cannock, England, United Kingdom', '30 days returns', 'Stock Must Go', 'New')
('313831486251', "Netac Internal SSD 2.5'' SATA III M.2 NVMe PCIe Gen 3.0×4 Solid State Drive lot", 'Memory Drives & Storage', 'More than 10 available', '2,283 ', 'US $18.99', 'IDR283,432.84', 'US $20.99 ', 'Hebro

In [10]:
print(len(dataArr))

for i in range(5):
    PrintTupleData(dataArr[i])

243
354907151520 || Microsoft 13" Surface Pro 8 (i5, 8GB RAM, 256GB SSD, Graphite) with Surface Pen || Laptops & Netbooks || Limited quantity available || 1  || US $669.99 || IDR9,999,850.75 || US $54.46  || Avenel, New Jersey, United States || 30 days returns || Always Deals Store || New || 
333563280818 || CHUWI Laptop HeroBook GemiBook CoreBook Pro/Plus/X Intel Core Laptop Windows || Laptops & Netbooks || Limited quantity available || 32  || EUR 239.99 || IDR4,026,996.38 || Free || None || 30 days returns || CHUWI Official Store（chuwi2018） || New || 
333996205343 || CHUWI HeroBook Pro 14.1" Windows 10 Laptop Intel N4020 Laptop 8+256GB || Laptops & Netbooks || Limited quantity available || 14  || EUR 199.99 || IDR3,355,802.35 || Free || None || 30 days returns || CHUWI Official Store（chuwi2018） || New || 
295671743764 || HP Omen 17.3" Gaming Notebook QHD 165Hz i9-13900HX 32GB RAM 1TB SSD RTX 4090 || Laptops & Netbooks || Limited quantity available || 92  || US $2,499.99 || IDR37,313,

# **DATAFRAME** (PANDAS AND NUMPY)

In [11]:
import numpy as np
import pandas as pd


# Making into data frame
numpyArr = np.array(dataArr)
df = pd.DataFrame(numpyArr)
df.columns = ["listing_id", "title", "type", "available", "sold", "price_original", "price_idr", "shipping_price_original", "shipping_location", "return", "seller", "condition"]
df

,listing_id,title,type,available,sold,price_original,price_idr,shipping_price_original,shipping_location,return,seller,condition
0,354907151520,"Microsoft 13"" Surface Pro 8 (i5, 8GB RAM, 256G...",Laptops & Netbooks,Limited quantity available,1,US $669.99,"IDR9,999,850.75",US $54.46,"Avenel, New Jersey, United States",30 days returns,Always Deals Store,New
1,333563280818,CHUWI Laptop HeroBook GemiBook CoreBook Pro/Pl...,Laptops & Netbooks,Limited quantity available,32,EUR 239.99,"IDR4,026,996.38",Free,None,30 days returns,CHUWI Official Store（chuwi2018）,New
2,333996205343,"CHUWI HeroBook Pro 14.1"" Windows 10 Laptop Int...",Laptops & Netbooks,Limited quantity available,14,EUR 199.99,"IDR3,355,802.35",Free,None,30 days returns,CHUWI Official Store（chuwi2018）,New
3,295671743764,"HP Omen 17.3"" Gaming Notebook QHD 165Hz i9-139...",Laptops & Netbooks,Limited quantity available,92,"US $2,499.99","IDR37,313,283.58",None,"Atlanta, Georgia, United States",30 days returns,Antonline,New
4,285185433173,"Lenovo ThinkPad P1 Gen 5 15.6"" 2022 2K 2.3GHz ...",Laptops & Netbooks,Limited quantity available,5,"US $1,399.99","IDR20,895,373.13",US $168.15,"Sanford, Florida, United States",30 days returns,ItsWorthMore,Excellent - Refurbished
...,...,...,...,...,...,...,...,...,...,...,...,...
238,313784767380,Smart Wireless WiFi Ring Doorbell Security Int...,Other Consumer Electronics,More than 10 available,949,US $33.99/ea,"IDR507,313.43",US $24.71,"La Puente, California, United States",30 days returns,Whizzohead,New
239,285164489538,BOSS Audio Systems UNI4RGB UTV Marine Stereo P...,Other Consumer Electronics,More than 10 available,1,US $400.99,"IDR5,984,925.37",US $55.12,"Oxnard, California, United States",30 days returns,SpeeceInc,New
240,283619457023,BOSS Audio Systems MR692B 6 x 9 Inch Marine St...,Other Consumer Electronics,More than 10 available,87,US $65.00,"IDR970,149.25",US $58.89,"Oxnard, California, United States",30 days returns,SpeeceInc,New
241,284347290788,BOSS Audio Systems MRWT69 6 x 9 ATV UTV Marine...,Other Consumer Electronics,More than 10 available,81,US $109.88,"IDR1,640,000.00",US $69.19,"Oxnard, California, United States",30 days returns,SpeeceInc,New


In [12]:
import os
dir = os.getcwd()
print(dir[:-3]+"data")
df.to_json(dir[:-3]+"data\data.json", orient='index', indent=2 )

d:\Folder Kuliah Cadangan\UKM and Others\Aslab\Basdat\Seleksi-2023-Tugas-1\Data Scraping\data
